### Instancia das libs

In [ ]:
import sys
import pandas as pd
from unidecode import unidecode

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

### Scrapper do site do Tesouro Direto

In [ ]:
url = 'https://www.tesourodireto.com.br/titulos/precos-e-taxas.htm'

In [ ]:
driver = webdriver.Chrome()
driver.get(url)

In [ ]:
# atualização

element_data_atualizacao = driver.find_element(By.XPATH, '/html/body/main/div[1]/div[1]/div/div[3]/p/span')
momento_atualizacao = element_data_atualizacao.get_attribute('innerHTML')
print('Ultima atualização:', momento_atualizacao)

In [ ]:
# titulos a venda

element_tab_titulos = driver.find_element(By.CLASS_NAME, 'td-mercado-titulos__content')
element_tab_titulos_html = element_tab_titulos.get_attribute('innerHTML')

try:
    df_todos_titulos = pd.read_html(element_tab_titulos_html)
    driver.close()
except:
    print("No table found")
    driver.close()
    sys.exit(0)

print('TABELA: INVESTIR')

df_todos_titulos[0]

### Tratamento dos dados

In [ ]:
new_columns_name = { 
    'Título':'titulo', 
    'Rentabilidade anual:':'rentabilidade_anual', 
    'Investimento mínimo:':'investimento_minimo',
    'Preço Unitário:':'preco_unitario',
    'Vencimento:':'vencimento'
}
df_titulos = df_todos_titulos[0].rename(columns=new_columns_name)
df_titulos.drop('Unnamed: 5', axis=1, inplace=True)
df_titulos.drop('Unnamed: 6', axis=1, inplace=True)

In [ ]:
def tipo_titulo(titulo):
    titulo = str(titulo).lower()
    if titulo.__contains__('prefixado'):
        return 'prefixado'
    elif titulo.__contains__('selic'):
        return 'posfixado'
    elif titulo.__contains__('ipca'):
        return 'inflacao'
    elif titulo.__contains__('igp-m'):
        return 'inflacao'
    elif titulo.__contains__('renda+'):
        return 'inflacao'

In [ ]:
df_titulos['tipo'] = df_titulos['titulo'].apply(tipo_titulo)

In [ ]:
df_titulos['juros_semestrais'] = df_titulos['titulo'].apply(lambda x: 'sim' if str(x).lower().__contains__('com juros semestrais') else 'nao')

In [ ]:
df_titulos['titulo'] = df_titulos['titulo'].apply(lambda x: str(x).lower())

In [ ]:
def limpa_taxa_remuneracao(rentabilidade):
    rentabilidade = str(rentabilidade)

    if rentabilidade.__contains__('%'):
        rentabilidade = rentabilidade.replace('%', '')

    if rentabilidade.__contains__('SELIC + '):
        rentabilidade = rentabilidade.replace('SELIC + ', '')

    if rentabilidade.__contains__('IPCA + '):
        rentabilidade = rentabilidade.replace('IPCA + ', '')

    if rentabilidade.__contains__('IGP-M + '):
        rentabilidade = rentabilidade.replace('IGP-M + ', '')
    
    if rentabilidade.__contains__(','):
        rentabilidade = rentabilidade.replace(',', '.')

    return round(float(rentabilidade), 2)

In [ ]:
df_titulos['taxa_remuneracao'] = df_titulos['rentabilidade_anual'].apply(limpa_taxa_remuneracao)

In [ ]:
def converte_preco_str_para_float(rentabilidade):
    rentabilidade = str(rentabilidade)

    if rentabilidade.__contains__('R$ '):
        rentabilidade = rentabilidade.replace('R$ ', '')
    
    if rentabilidade.__contains__('.'):
        rentabilidade = rentabilidade.replace('.', '')

    if rentabilidade.__contains__(','):
        rentabilidade = rentabilidade.replace(',', '')

    return round(float(rentabilidade)/100, 2)

In [ ]:
df_titulos['investimento_minimo'] = df_titulos['investimento_minimo'].apply(converte_preco_str_para_float)
df_titulos['preco_unitario'] = df_titulos['preco_unitario'].apply(converte_preco_str_para_float)

### Apresentação da tabela

In [ ]:
df_titulos

In [ ]:
df_titulos.to_csv('titulos_publicos.csv')